# Mini-Project I
During this project, we will practice handling of complex lists and dictionaries in Python. Plus, we will learn how to work with API documentation. Don't be afraid to search for the information in the [**documentation**](https://api.tfl.gov.uk/swagger/ui/index.html?url=/swagger/docs/v1#!/AccidentStats/AccidentStats_Get).

Let's go to the tasks, we have some parsing to do :)!!

In [1]:
# import packages we need (remember what packages we used yesterday during the API session)
import requests
from config import *
from datetime import datetime
import pandas as pd

## Demo

In [2]:
# don't forget package os
app_id = TFL_ID
app_key = TFL_SECRET
url_append = f'?app_id={app_id}&app_key={app_key}' 

In [3]:
# URL
url_airQuality = 'https://api.tfl.gov.uk/AirQuality'

In [4]:
# We send the request to the API
# NOTE: if you don't have your APP_KEY, run this without the url_append
res = requests.get(url_airQuality + url_append)

In [5]:
# We can check if the request was successful
res.status_code

200

In [6]:
# We can export the information that was returned using method .json()
res.json()

{'$id': '1',
 '$type': 'Tfl.Api.Presentation.Entities.LondonAirForecast, Tfl.Api.Presentation.Entities',
 'updatePeriod': 'hourly',
 'updateFrequency': '1',
 'forecastURL': 'http://londonair.org.uk/forecast',
 'disclaimerText': 'This forecast is intended to provide information on expected pollution levels in areas of significant public exposure. It may not apply in very specific locations close to unusually strong or short-lived local sources of pollution.',
 'currentForecast': [{'$id': '2',
   '$type': 'Tfl.Api.Presentation.Entities.CurrentForecast, Tfl.Api.Presentation.Entities',
   'forecastType': 'Current',
   'forecastID': '35073',
   'publishedDate': '2022-02-02T11:37:02Z',
   'fromDate': '2022-03-02T00:00:00Z',
   'toDate': '2022-04-02T23:59:00Z',
   'forecastBand': 'Low',
   'forecastSummary': 'Low air pollution forecast valid from Thursday 3 February to end of Friday 4 February GMT',
   'nO2Band': 'Low',
   'o3Band': 'Low',
   'pM10Band': 'Low',
   'pM25Band': 'Low',
   'sO2Ba

## Task
Parse the dictionary and print the AirQuality predictions for tomorrow

In [7]:
print(f'AirQuality prediction for tomorrow:\n{res.json()["currentForecast"][0]["forecastSummary"]}')

AirQuality prediction for tomorrow:
Low air pollution forecast valid from Thursday 3 February to end of Friday 4 February GMT


## Task
What are the different modes of transport which are operated by Transport for London? How many of modes do they have?

Print the list with different modes of transport, plus their count. Example output:
```
[bus, cable-car,.....]
Number of different modes of transport is: xyz
```

We need to search the documentation for correct request.

In [16]:
res_modes = requests.get(f'https://api.tfl.gov.uk/Journey/Meta/Modes{url_append}')

In [20]:
list_modes = [mode['modeName'] for mode in res_modes.json()]
print(f'{list_modes}\nNumber of different modes of transport is: {len(list_modes)}')

['black-cab-as-customer', 'black-cab-as-driver', 'bus', 'cable-car', 'coach', 'cycle', 'cycle-hire', 'dlr', 'electric-car', 'goods-vehicle-as-driver', 'interchange-keep-sitting', 'interchange-secure', 'international-rail', 'motorbike-scooter', 'national-rail', 'overground', 'plane', 'private-car', 'private-coach-as-customer', 'private-coach-as-driver', 'private-hire-as-customer', 'private-hire-as-driver', 'replacement-bus', 'river-bus', 'river-tour', 'taxi', 'tflrail', 'tram', 'tube', 'walking']
Number of different modes of transport is: 30


## Task
How many BikePoints in London are operated by Transport for London? How many docks are in **all** BikePoints? There is the information for empty and full docks for each BikePoint.

In [22]:
res_bikePoints = requests.get(f'https://api.tfl.gov.uk/BikePoint{url_append}')

In [ ]:
list_dock = []
for point in res_bikePoints.json():
    for prop in point['additionalProperties']:
        if 'Dock' in prop['key']:
            list_dock.append(prop['key'])

In [41]:
list_bikePoints = [res_bikePoints.json()[i]['id'] for i in range(len(res_bikePoints.json()))]
print(f'Total BikePoints in London operated by Transfor for London: {len(list_bikePoints)}')
print(f'Total docks in all BikePoints {len(list_dock)}')

Total BikePoints in London operated by Transfor for London: 787
Total docks in all BikePoints 1574


## Task
How many tube and bus lines are in London? Print names of all tube lines.

In [42]:
res_tube = requests.get(f'https://api.tfl.gov.uk/Line/Mode/tube{url_append}')
res_bus = requests.get(f'https://api.tfl.gov.uk/Line/Mode/bus{url_append}')

In [61]:
total_lines = len(res_tube.json()) + len(res_bus.json())
names_tube = [line['name'] for line in res_tube.json()]
print(f'Total Lines: {total_lines}')
print(f'Names of all tube lines:\n{names_tube}')

Total Lines: 688
Names of all tube lines:
['Bakerloo', 'Central', 'Circle', 'District', 'Hammersmith & City', 'Jubilee', 'Metropolitan', 'Northern', 'Piccadilly', 'Victoria', 'Waterloo & City']


## Task
How many station has `victoria` line?

In [62]:
res_stat_vic = requests.get(f'https://api.tfl.gov.uk/Line/victoria/StopPoints{url_append}')

In [68]:
total_stat_vic = len(res_stat_vic.json())
print(f'Stations that have victoria line: {total_stat_vic}')

Stations that have victoria line: 16


## Task
Plan the journey from Heathrow Airport to Tower Bridge using Bus and Tube? Which way is faster? Example output:
```
Planned duration:
Bus: x minutes
Tube: y minutes
```

We need to search the documentation for correct requests and parameters we need.

In [109]:
heathrowAirpot = '51.4700223,-0.4564735'
towerBridge = '51.4717691,-0.1205621'
by_bus = requests.get(f'https://api.tfl.gov.uk/Journey/JourneyResults/{heathrowAirpot}/to/{towerBridge}?mode=bus')
by_tube = requests.get(f'https://api.tfl.gov.uk/Journey/JourneyResults/{heathrowAirpot}/to/{towerBridge}?mode=tube')

In [110]:
travelTime_bus = by_bus.json()['journeys'][0]['duration']
travelTime_tube = by_tube.json()['journeys'][0]['duration']

print('Planned duration:')
print(f'Bus: {travelTime_bus} mins')
print(f'Tube: {travelTime_tube} mins')

Planned duration:
Bus: 172 mins
Tube: 76 mins
